# 规则验证模块

## 主要任务

规则验证模块的主要任务，是将硬匹配和软匹配的结果进行二次打标来验证。因为规则的标注并非完全准确，原本需要人去验证，但是人力验证成本过高，因此使用模型来代替人进行验证。

这个模块需要一些训练语料来训练模型，同时需要上一轮的软硬匹配结果来进行二次打标。

## 具体流程

1. 模型训练
2. 模型评估
3. 模型预测 / 句子打标
4. 冲突检验


# 实验数据准备

将 sent_train 数据集中的句子和标签全部利用来训练模型

利用 Word2vec 将句子变成向量

In [3]:
# 导入实验数据
import pandas as pd

train_data = pd.read_csv("/Users/xuhaoshuai/GitHub/HumanIE_IPM/data/sent_train/brother.csv")
train_data

,sent,label
0,四 薛 人物二 人物一 薛宝钗 薛宝琴,1
1,冷子兴 官 人物二 人物一 外出 做生意,1
2,冷子兴 官 人物二 人物一 邢岫烟 外出 做生意 参与 贩卖 官盐 邢岫烟 并不知道 情况,1
3,人物二 打死 人命 牢里 耐不住 寂寞 勾引 人物一,1
4,大色 空四 薛 人物二 人物一 薛宝钗 薛宝琴,1
...,...,...
995,人物一 问知 人物二 曰 务民之义 敬鬼神而远之 可谓 知矣,0
996,人物一 请学稼 人物二 曰 吾 老农,0
997,学生 人物一 问 智 人物二 答 知 论语 颜渊,0
998,人物二 力倡 爱人 人物一 问 仁 人物二 回答 爱人 论语 颜渊,0


In [6]:
# 抽取句子部分
train_sent_set = train_data['sent']

train_sent_set

0                                    四 薛 人物二 人物一 薛宝钗 薛宝琴
1                                   冷子兴 官 人物二 人物一 外出 做生意
2          冷子兴 官 人物二 人物一 邢岫烟 外出 做生意 参与 贩卖 官盐 邢岫烟 并不知道 情况
3                             人物二 打死 人命 牢里 耐不住 寂寞 勾引 人物一
4                                大色 空四 薛 人物二 人物一 薛宝钗 薛宝琴
                             ...                        
995                       人物一 问知 人物二 曰 务民之义 敬鬼神而远之 可谓 知矣
996                                   人物一 请学稼 人物二 曰 吾 老农
997                             学生 人物一 问 智 人物二 答 知 论语 颜渊
998                    人物二 力倡 爱人 人物一 问 仁 人物二 回答 爱人 论语 颜渊
999     人物二 弟子 冉有 没 老师 失望 带 周父 人物一 积极 备战 派 武城 三百 守住 宫...
Name: sent, Length: 1000, dtype: object

In [ ]:
# 调用 Word2vec
from gensim.models import Word2Vec, word2vec

model = Word2Vec.load("/Users/xuhaoshuai/GitHub/HumanIE_IPM/src/softmatch/word2vec.model")